In [2]:
import pandas as pd
import datetime
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pickle

In [3]:
train1 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
train = train[train.notnull().apply(all, axis=1)]
dev = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_dev.csv')
dev = dev[dev.notnull().apply(all, axis=1)]
test = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_test.csv')
test = test[test.notnull().apply(all, axis=1)]

dev_prepared = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrase_vectors_dev.csv', sep=';')
dev_prepared = dev_prepared.drop(columns='Unnamed: 0')
dev = dev.set_index('pairID')
rel_pairIDs = dev_prepared.iloc[:,0]
y_hat = dev.loc[rel_pairIDs].gold_label
dev_prepared = dev_prepared.iloc[:,1:].to_numpy()

In [4]:
data1 = pd.read_csv("prepared_data/GPT3_explanations1.csv", sep=";")
data2 = pd.read_csv("prepared_data/GPT3_explanations2.csv", sep=";")

In [5]:
temp = [None, ] * 10
for i in range(1,11):
    temp[i-1] = pd.read_csv("prepared_data/GPT3_explanations" + str(i) + ".csv", sep=";")
gpt3_dev = pd.concat(temp).set_index("pairID")

In [6]:
np.mean(gpt3_dev.loc[rel_pairIDs].gold_standard_label == gpt3_dev.loc[rel_pairIDs].pred_label)

0.8069192751235584

In [7]:
f1_score(gpt3_dev.loc[rel_pairIDs].gold_standard_label, gpt3_dev.loc[rel_pairIDs].pred_label, average="macro")

0.7884484917636336

In [18]:
gpt3_dev.loc[rel_pairIDs].iloc[np.where(gpt3_dev.loc[rel_pairIDs].gold_standard_label == gpt3_dev.loc[rel_pairIDs].pred_label)]

,Unnamed: 0,gold_standard_explanation,gold_standard_label,prompts_instruct_GPT,pred_explanation,pred_label
pairID,,,,,,
4705552913.jpg#2r1e,1,Saying the two women are holding packages is a...,entailment,"Classify into entailment, neutral, or contradi...",Embracing is a form of holding.,entailment
4705552913.jpg#2r1c,2,In the first sentence there is an action of af...,contradiction,"Classify into entailment, neutral, or contradi...",Embracing and fighting are different actions.,contradiction
3948003394.jpg#1r1e,18,Both woman are doing a cartwheel.,entailment,"Classify into entailment, neutral, or contradi...",A woman doing a cartwheel is wearing a bikini ...,entailment
3948003394.jpg#1r1c,19,It is not feasible to be in the process of fix...,contradiction,"Classify into entailment, neutral, or contradi...",A woman doing a cartwheel in a bikini in the s...,contradiction
5436250638.jpg#4r1c,42,The subjects cannot perform surgery and be hav...,contradiction,"Classify into entailment, neutral, or contradi...",Two doctors performing surgery cannot be havin...,contradiction
...,...,...,...,...,...,...
6087966524.jpg#2r1e,9827,Two police officers can be rephrased as The Po...,entailment,"Classify into entailment, neutral, or contradi...",The premise states that the police officers ar...,entailment
3344632789.jpg#0r2c,9828,Performing an aerial skateboard trick and orde...,contradiction,"Classify into entailment, neutral, or contradi...",A man performing an aerial skateboard trick on...,contradiction
3344632789.jpg#0r1e,9831,Several people are people.,entailment,"Classify into entailment, neutral, or contradi...",The premise states that there are several peop...,entailment


In [39]:
print(gpt3_dev.iloc[0].prompts_instruct_GPT)

Classify into entailment, neutral, or contradiction and justify the decision.

Premise: Three musicians in a recording studio.
Hypothesis: Folks making music.
Label: entailment
Explanation: Three musicians is a rephrasing of folks. If they are in a recording studio, they would be making music.
###
Premise: A man sits on the sidewalk in front of an Asian restaurant as people pass.
Hypothesis: Man eating noodles
Label: contradiction
Explanation: eating noodles is not usually done when one sits on the sidewalk.
###
Premise: A middle-aged lady holds a young girl in her arms as they sleep in an indoor booth.
Hypothesis: A middle-aged lady and young girl window shop.
Label: contradiction
Explanation: The lady and the girl cannot shop if they are asleep.
###
Premise: A young, left handed batter in baseball the moment after he made a hit with a catcher nearby and a crowd in the background.
Hypothesis: The batter is a boy.
Label: entailment
Explanation: The young, left handed better is a boy.
#

In [35]:
gpt3_dev.index[1235]

'4345676156.jpg#4r1e'